In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tqdm.keras import TqdmCallback  # For progress bar

# -----------------------------
# Define dataset paths
# -----------------------------
dataset_dir = r"C:\Users\okeiy\OneDrive - University of Salford\Documents\Sch Notes\Dissertation\Codes\Carla_dataset"
csv_path = os.path.join(dataset_dir, "control_commands.csv")
images_dir = os.path.join(dataset_dir, "recorded_images")

# -----------------------------
# Load CSV file
# -----------------------------
data = pd.read_csv(csv_path)
print("Dataset loaded, sample:")
print(data.head())

# -----------------------------
# Split into training and validation sets
# -----------------------------
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

# -----------------------------
# Preprocessing Functions
# -----------------------------
def preprocess_image(image_path):
    """
    Reads an image from disk, converts from BGR to RGB,
    resizes to (66, 200) as required by PilotNet, and normalizes pixel values.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found: " + image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (200, 66))  # width, height
    img = img.astype(np.float32) / 255.0
    return img

def load_data(row):
    """
    Given a CSV row, load the corresponding image and extract control signals.
    """
    image_path = row['image_filename']
    if not os.path.isabs(image_path):
        image_path = os.path.join(images_dir, os.path.basename(image_path))
    image = preprocess_image(image_path)
    labels = row[['steer', 'throttle', 'brake']].values.astype(np.float32)
    return image, labels

def generator(df):
    """
    Generator function that yields (image, labels) tuples.
    """
    for _, row in df.iterrows():
        try:
            yield load_data(row)
        except Exception as e:
            print(f"Error loading row: {e}")
            continue

# -----------------------------
# Create tf.data.Dataset objects using output_signature
# -----------------------------
train_ds = tf.data.Dataset.from_generator(
    lambda: generator(train_df),
    output_signature=(
        tf.TensorSpec(shape=(66, 200, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(3,), dtype=tf.float32)
    )
)
val_ds = tf.data.Dataset.from_generator(
    lambda: generator(val_df),
    output_signature=(
        tf.TensorSpec(shape=(66, 200, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(3,), dtype=tf.float32)
    )
)

batch_size = 32
# Use .repeat() so that the training dataset loops indefinitely
train_ds = train_ds.repeat().shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Calculate steps per epoch based on the number of training samples
steps_per_epoch = int(np.ceil(len(train_df) / batch_size))
validation_steps = int(np.ceil(len(val_df) / batch_size))

# -----------------------------
# Build the NVIDIA PilotNet Model (Functional API)
# -----------------------------
inputs = tf.keras.Input(shape=(66, 200, 3))
x = layers.Conv2D(24, (5, 5), strides=(2, 2), activation='relu')(inputs)
x = layers.Conv2D(36, (5, 5), strides=(2, 2), activation='relu')(x)
x = layers.Conv2D(48, (5, 5), strides=(2, 2), activation='relu')(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(100, activation='relu')(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dense(10, activation='relu')(x)
outputs = layers.Dense(3, activation='linear')(x)  # steer, throttle, brake
model = models.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mse')
model.summary()

# -----------------------------
# Train the Model with TQDM Progress Bar
# -----------------------------
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[TqdmCallback(verbose=1)]
)

# -----------------------------
# Save the Trained Model
# -----------------------------
model_save_path = os.path.join(dataset_dir, "pilotnet_model.h5")
model.save(model_save_path)
print("Model saved to:", model_save_path)

Dataset loaded, sample:
   frame   timestamp                                     image_filename  \
0      0  211.801474  C:\Users\okeiy\OneDrive - University of Salfor...   
1      1  211.851474  C:\Users\okeiy\OneDrive - University of Salfor...   
2      2  211.901474  C:\Users\okeiy\OneDrive - University of Salfor...   
3      3  211.951474  C:\Users\okeiy\OneDrive - University of Salfor...   
4      4  212.001474  C:\Users\okeiy\OneDrive - University of Salfor...   

      steer  throttle  brake  
0 -0.001231      0.85    0.0  
1  0.002093      0.85    0.0  
2  0.003942      0.85    0.0  
3  0.003942      0.85    0.0  
4  0.004526      0.85    0.0  


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 66, 200, 3)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 31, 98, 24)          │           1,824 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 14, 47, 36)          │          21,636 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 5, 22, 48)           │          43,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 3, 20, 64)           │          27,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 1, 18, 64)           │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 100)                 │         115,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             510 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 252,241 (985.32 KB)

 Trainable params: 252,241 (985.32 KB)

 Non-trainable params: 0 (0.00 B)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - loss: 0.0869 - val_loss: 0.0891
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - loss: 0.0827 - val_loss: 0.0874
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: 0.0748 - val_loss: 0.0785
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: 0.0608 - val_loss: 0.0433
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: 0.0438 - val_loss: 0.0334
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: 0.0323 - val_loss: 0.0294
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: 0.0384 - val_loss: 0.0157
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: 0.0150 - val_loss: 0.0100
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: 0.0115 - val_loss: 0.0063
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: 0.0086 - val_loss: 0.0054


Model saved to: C:\Users\okeiy\OneDrive - University of Salford\Documents\Sch Notes\Dissertation\Codes\Carla_dataset\pilotnet_model.h5
